In [57]:
pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [58]:
import sqlalchemy
from urllib import parse
import pandas as pd
import pymysql

In [59]:
user = 'swp'
password = 'swp1'
host='172.25.249.254'
port = 3306
database = 'classicmodels'
password = parse.quote_plus(password)
engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")

con =pymysql.connect(host='172.25.249.254', user='swp', password='swp1', db='classicmodels', charset='utf8')
cur = con.cursor()


In [18]:
order = pd.read_sql_query("select * from orders", con=engine)
orderdetails = pd.read_sql_query("select * from orderdetails", con=engine)
customers = pd.read_sql_query("select * from customers", con=engine)

In [20]:
df = pd.merge(pd.merge(order, orderdetails, on='orderNumber', how='left'), customers, on="customerNumber", how='left')

In [21]:
df['tmp'] = df['priceEach'] * df['quantityOrdered']

In [25]:
df.groupby(['country', 'city'], as_index=False)[['tmp']].sum().sort_values(by=['country', 'city']).rename(columns={'tmp' : 'sales'})

,country,city,sales
0,Australia,Chatswood,133907.12
1,Australia,Glen Waverly,55866.02
2,Australia,Melbourne,180585.07
3,Australia,North Sydney,137034.22
4,Australia,South Brisbane,55190.16
...,...,...,...
73,USA,San Diego,80375.24
74,USA,San Francisco,199051.34
75,USA,San Jose,143536.27
76,USA,San Rafael,591827.34


In [27]:
df2 = df.groupby(['country', 'city'], as_index=False)[['tmp']].sum(). \
        sort_values(by=['country', 'city']).rename(columns={'tmp' : 'sales'})

In [34]:
tmp = pd.DataFrame({'a' : [55,45,66,22,45,88,99,100]})

In [36]:
tmp['a'].rank()

0    4.0
1    2.5
2    5.0
3    1.0
4    2.5
5    6.0
6    7.0
7    8.0
Name: a, dtype: float64

In [32]:
df2['rank'] = df2['sales'].rank(ascending=False)

In [33]:
df2.sort_values(by=['sales'], ascending=False)

,country,city,sales,rank
46,Spain,Madrid,979880.77,1.0
76,USA,San Rafael,591827.34,2.0
66,USA,NYC,497941.50,3.0
38,New Zealand,Auckland,292082.87,4.0
44,Singapore,Singapore,263997.78,5.0
...,...,...,...,...
59,USA,Brisbane,42570.37,74.0
65,USA,Los Angeles,41506.19,75.0
28,Germany,Munich,31310.09,76.0
8,Belgium,Charleroi,29217.18,77.0


In [38]:
import sqlalchemy


In [40]:
data = pd.read_csv('./data/dataset2.csv', delimiter=';')

In [41]:
data.shape

(20603, 10)

In [42]:
display(data.head(1))
print(data.columns)

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates


Index(['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')


In [47]:
for x in data.columns:
    print(f"`{x}`")

Clothing ID
Age
Title
Review Text
Rating
Recommended IND
Positive Feedback Count
Division Name
Department Name
Class Name


In [83]:
sql = """CREATE TABLE dataset2
(
    `Clothing ID` INT,
    `Age` INT,
    `Title` VARCHAR(200), 
    `Review Text` TEXT,
    `Rating` INT,
    `Recommended IND` INT,
    `Positive Feedback Count` INT,
    `Division Name` VARCHAR(30), 
    `Department Name` VARCHAR(30), 
    `Class Name` VARCHAR(30), 
)"""

In [84]:
cur.execute(sql)

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ')' at line 13")

In [48]:
def tmp(x):
    print(x)

In [85]:
def tmp(x):
    cur.execute("insert into dataset2 values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", x.tolist())

In [74]:
data.head(1)

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,None,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates


In [86]:
data.apply(tmp,axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
20598    None
20599    None
20600    None
20601    None
20602    None
Length: 20603, dtype: object

In [76]:
data = data.where(pd.notnull(data), None)

In [77]:
cur.execute("show tables")

13

In [78]:
cur.fetchall()

(('customers',),
 ('dataset2',),
 ('employees',),
 ('offices',),
 ('orderdetails',),
 ('orders',),
 ('payments',),
 ('productlines',),
 ('products',),
 ('stat',),
 ('subway',),
 ('tmp',),
 ('tmp_orders',))

In [61]:
cur.execute('truncate table dataset2')

0

In [62]:
con.commit()

In [79]:
insert_sql = "insert into dataset2 values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

In [82]:
data.apply(lambda x : cur.execute(insert_sql, x.tolist()), axis=1)

0        1
1        1
2        1
3        1
4        1
        ..
20598    1
20599    1
20600    1
20601    1
20602    1
Length: 20603, dtype: int64

In [81]:
con.commit()

In [88]:
data.groupby(['Division Name'])[['Rating']].mean().sort_values(by=['Rating'], ascending=True)

,Rating
Division Name,
General,4.172644
General Petite,4.208867
Initmates,4.294985


In [92]:
def tmp(x):
    if x < 20:
        return "청소년"
    elif x < 50:
        return "청년"
    else:
        return "중년"

In [94]:
data['Age'].apply(tmp)

0        청년
1        청년
2        중년
3        중년
4        청년
         ..
20598    청년
20599    청년
20600    청년
20601    청년
20602    중년
Name: Age, Length: 20603, dtype: object

In [97]:
data['age_class'] = data['Age'].apply(lambda x : '청소년' if x < 20 else ("청년" if x < 50 else "중년"))

In [98]:
data['age_class'].value_counts()

age_class
청년     14730
중년      5833
청소년       40
Name: count, dtype: int64

In [ ]:
select case when age between 0 and 19 then '청소년'
when age between 20 and 49 then '청년'
when age between 49 and 99 then '중년' end age_class
from dataset2;